# Handwritten Digit Recognition with Convolutional Neural Nets


## Setup

In [ ]:
import tensorflow as tf
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

In [ ]:
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

In [ ]:
# utility functions
def show_images(images, min_width = 0):
    images = np.reshape(images, [images.shape[0], -1])  # images reshape to (batch_size, D)
    sqrtn = int(np.ceil(np.sqrt(images.shape[0])))
    sqrtimg = int(np.ceil(np.sqrt(images.shape[1])))
    
    sqrtn = max(min_width, sqrtn)
        
    fig = plt.figure(figsize=(sqrtn, sqrtn))
    gs = gridspec.GridSpec(sqrtn, sqrtn)
    gs.update(wspace=0.05, hspace=0.05)

    for i, img in enumerate(images):
        ax = plt.subplot(gs[i])
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
        plt.imshow(img.reshape([sqrtimg,sqrtimg]))
    return

def visualise_errors(incorrect_classifications):
    for digit_class in range(10):        
        sample = [class_prototype[digit_class]] + incorrect_classifications[digit_class][:15]            
        show_images(X_test[sample,:], 16)        

## Dataset

We will be working on the MNIST dataset, which is 60,000 training and 10,000 test images. Each picture contains a centered image of white digit on black background (0 through 9). This was one of the first datasets used to train convolutional neural networks and it is fairly easy -- a standard CNN model can easily exceed 99% accuracy. 

To simplify our code here, we will use the TensorFlow MNIST wrapper, which downloads and loads the MNIST dataset. See the [documentation](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/contrib/learn/python/learn/datasets/mnist.py) for more information about the interface. The default parameters will take 5,000 of the training examples and place them into a validation dataset. The data will be saved into a folder called `MNIST_data`. 


In [ ]:
# Load the dataset

# k-Nearest Neigbours (kNN)

The idea behind kNN is to classify a digit by looking at its 'closest' analogy.  That is, to compare an unknown digit to a set of known digits.  

Some problems can be solved quite well by this deceptively simple algorithm, especially if a good distance metric is used.  

The 'k' in kNN refers the number of closest examples to consider.  For example, k=5 would mean to take the 5 most similar digits and pick the most common one.

Each problem will have a different best k value.  But 3 or 5 usually works quite well.

kNN is also great at providing a **baseline**.  A more advanced model performing worse than kNN is a strong indicator of a bug in your algorithm.


### Split the test and training data

The goal is to learn how to predict **y** from **X**.  Where **X** is is the image, and **y** is the label (i.e. 0,1,2...)

We will use the training data to learn how to predict digits, and the test data to check how well the model is working.  It is important that the model never sees the test data.

In [ ]:
# Take a (n=5000) training sample, and a (n=500) test sample.

### Setup the Model

Now we setup our model in TensorFlow.  At this point, *we are not doing any of the calculations*.  We are simply creating a graph that will be used during training.  You can think of this as creating a 'blueprint' describing what calculations to perform during training.

TensorFlow operations can be performed using method calls, e.g. c = tf.add(a,b), or by using operators, e.g. c = a + b

We will also make use of the tf.nn module which contains some very useful support functions for neural nets.

The following code was modified from the [tensorflow examples](https://github.com/aymericdamien/TensorFlow-Examples/blob/master/examples/2_BasicModels/nearest_neighbor.py)

In [ ]:
# Setup the model

### Train

The kNN algorithm doesn't really need to train, as it simply memorises the training data.  So at this point we are simply going to evaluate the accuracy of the model.

In [ ]:
# Train the model

### Inspecting the miss clasified examples

It's always a good idea to take a look at what kinds of errors the algorithm is making.

In [ ]:
# Display some missclassifed examples

# 3. Convolutional Neural Net

### What is a Convolutional Neural Net?


In a standard neural net, each neuron responds based on a linear combination of inputs from the previous layer.

<img src="http://cs231n.github.io/assets/nn1/neural_net2.jpeg" width = 40%>

In a convolutional neural net, each neuron is a 'filter' which responds to different kinds of 2d patterns.  At the base layers, these will simply be edge detectors, but higher layers neurons will trigger on more abstract concepts.  Each filter is run over the entire image and outputs a new image with the applied filter.  These filtered images are then 'stacked' on top of each other to create a 3D volume used as the input to the next layer.

<img src="http://cs231n.github.io/assets/cnn/cnn.jpeg", width = 40%>

We add pooling layers that reduce the size of the image.  The idea being that at deeper layers the exact details are not needed, just the general ideas.  Typically these pooling layers reduce the size of the image by 2 in each direction.

<img src="http://cs231n.github.io/assets/cnn/convnet.jpeg">


See the [CS231n](http://cs231n.github.io/convolutional-networks/) site for more information.

Note: The following code was modified from [this](https://www.tensorflow.org/tutorials/layers) TensorFlow example.

## Single layer CCN

In [ ]:
# This time we will use the entire dataset.
X_train, y_train = mnist.train.images, mnist.train.labels
X_test, y_test = mnist.test.images, mnist.test.labels
class_prototype = [np.argmax(y_test, axis = 1).tolist().index(digit_class) for digit_class in range(10)]

In [ ]:
# Helper functions for training / evaluation.
def eval_model():
    print("-"*60)
    test_accuracy = accuracy.eval(feed_dict={X: mnist.test.images, y: mnist.test.labels, keep_prob: 1.0})
    print("Test Error Rate {0:.2f}%".format((1-test_accuracy)*100))
    
    predictions = pred.eval(feed_dict={X: mnist.test.images, y: mnist.test.labels, keep_prob: 1.0})
    pred_class = np.argmax(predictions, axis = 1)
    true_class = np.argmax(mnist.test.labels, axis = 1)
    
    incorrect_classifications = [[] for _ in range(10)]
        
    for i in range(true_class.shape[0]):
        if pred_class[i] != true_class[i]:
            incorrect_classifications[pred_class[i]].append(i)
            
    visualise_errors(incorrect_classifications)    
    

# train the model...
def train_model(epocs = 10, batch_size = 64):

    print("Training...")

    init = tf.global_variables_initializer()
    
    iterations = int(epocs * len(mnist.train.labels) / batch_size)

    # Start training
    with tf.Session() as sess:

        # Run the initializer
        sess.run(init)

        for i in range(iterations):

            # get a new batch
            batch = mnist.train.next_batch(batch_size)            

            # evaluate every 100 steps
            if i % 1000 == 0:
                val_batch = mnist.validation.next_batch(1000)
                train_batch = mnist.validation.next_batch(1000)

                train_accuracy = accuracy.eval(feed_dict={X: train_batch[0], y: train_batch[1], keep_prob: 1.0})
                val_accuracy = accuracy.eval(feed_dict={X: val_batch[0], y: val_batch[1], keep_prob: 1.0}) 

                loss_eval = loss.eval(feed_dict={X: batch[0], y: batch[1], keep_prob: 0.5})            
                
                epoc = (batch_size * i) / len(mnist.train.labels)

                print('[epoc=%g] step %d, training accuracy %g validation accuracy %g loss %g' % (epoc, i, train_accuracy, val_accuracy, loss_eval))        

            # train on this batch
            train_op.run(feed_dict={X: batch[0], y: batch[1], keep_prob: 0.5})

        eval_model()

### Setup the Model

In [ ]:
# Setup the model

In [ ]:
train_model()

## Two layer CCN

Two layer model based on [LeNet-5](http://yann.lecun.com/exdb/lenet/ )

See (http://yann.lecun.com/exdb/publis/pdf/lecun-01a.pdf)

In [ ]:
# Setup the model

In [ ]:
train_model()